In [147]:
import numpy as np
from tqdm import tqdm
from functools import reduce

In [2]:
with open("./data_inputs/day19_input.txt") as f:
    input = f.read().split("\n")

In [3]:
c = 0
scanners = []
for l in input:
    if l[0:3] == "---":
        scanner = []
    elif len(l) == 0:
        scanners.append(np.array(scanner))
    else:
        scanner.append(list(map(int, l.split(","))))



In [6]:
def internal_beacons_pattern(scanner):
    
    pattern = np.array([np.array([np.sqrt(np.sum((scanner_i - scanner_j)**2)) for scanner_j in scanner]) for scanner_i in scanner])  
    
    return pattern

In [7]:
patterns = [internal_beacons_pattern(scanner) for scanner in scanners]      # patterns 

In [158]:
print("scanners len:", len(scanners))
print("scanner0 beacons:", scanners[0].shape)
print("28 x 26 x 3")
print("\npatterns len:", len(patterns))
print("pattern0 shape:", patterns[0].shape, "26 distances for every single beacon of 26 beacons of the scanner 0 ")
print("28 x 26 x 26")

scanners len: 28
scanner0 beacons: (26, 3)
28 x 26 x 3

patterns len: 28
pattern0 shape: (26, 26) 26 distances for every single beacon of 26 beacons of the scanner 0 
28 x 26 x 26


In [27]:
coincidents = {}

for i, p1 in tqdm(enumerate(patterns)):
    for j, p2 in enumerate(patterns):
        if i >= j:
            continue

        coincident = np.zeros((p1.shape[0], p2.shape[0]))

        for k, b1 in enumerate(p1):
            for l, b2 in enumerate(p2):
                
                coincident[k, l] = sum([dist_b1 in b2 for dist_b1 in b1])

        coincidents[(i, j)] = coincident
        


28it [00:20,  1.36it/s]


In [130]:
beacons = []

for key_c, coincident in coincidents.items():
    coincident_b = np.array(np.where(coincident > 3)).T
    if coincident_b.size > 0:
        for beacon in coincident_b:
            beacons.append(((key_c[0], beacon[0]), (key_c[1], beacon[1])))

    

In [132]:
beacons_list_sets = [{beacons[0][0], beacons[0][1]}]

for beacon in beacons:
    found = False
    for beacon_set in beacons_list_sets:
        if beacon[0] in beacon_set:
            found = True
            beacon_set.add(beacon[1])
            break
        elif beacon[1] in beacon_set:
            found = True
            beacon_set.add(beacon[0])
            break
       
    if not found:
        beacons_list_sets.append({beacon[0], beacon[1]})



In [136]:
for i, beacon_set_i in enumerate(beacons_list_sets):
    for beacon in beacon_set_i:
        for j, beacon_set_j in enumerate(beacons_list_sets):
            if i == j:
                continue

            if beacon in beacon_set_j:
                beacon_set_i.update(beacon_set_j)
                beacons_list_sets.pop(j)


In [149]:
all_beacons = [[(i, j) for j, beacon in enumerate(scanner)] for i, scanner in enumerate(scanners)]
all_beacons = reduce(lambda x, y: x + y, all_beacons)

In [155]:
for beacon in all_beacons:
    found = False
    for beacon_set in beacons_list_sets:
        if beacon in beacon_set:
            found = True
            break

    if not found:
        beacons_list_sets.append({beacon})

In [157]:
print("Result1:", len(beacons_list_sets))

Result1: 362
